## Mini-tutorial: Creación de redes con Graph-Tool

Este mini-tutorial tiene como objetivo mostrar la creación de redes y la asignación de propiedades a la red, enlaces y vértices, usando el paquete _graph-tool_ de _python_.

Consiste en las siguientes secciones:

0. Preparación del ambiente
1. Creación de red: Caso simple
2. Creación de red: Enlaces con peso
3. Creación de red: Más propiedades
4. Asignación de nuevas propiedades
5. Salvar y cargar la red
6. Trabajando con propiedades

***

### 0. Preparación del ambiente
Lo primero que haremos es cargar las librerías _graph-tool_ y _pandas_.

_pandas_ es una librería de _python_ para manejar tablas de datos (_Dataframes_), la cual tiene utilidades que nos serán prácticas a la hora de seleccionar datos para la creación de redes.

In [ ]:
import pandas as pd
import numpy as np
from itertools import izip

# Cargamos paquete "Graph-tool" (version recomendada)
import graph_tool.all as gt

A continuación, cargamos los datos de ejemplo:

In [ ]:
Data = pd.read_csv('ejemploData.csv',dtype={'dia': str, 'hora': str})

# Echamos un vistazo a los primeros 15 registros de los datos
Data.head(15)

Vemos que hay tanto _CDRs_ de "El Melon" como de "Maria Elena", y estan mezclados.

Como mencionamos anteriormente, una de las ventajas de usar _pandas_ es que nos permite filtrar los datos según distintos criterios, lo cual es muy práctico al seleccionar datos, sobre todo dentro de ciclos (Ej.: Al crear redes por ciudades).
Veamos algunos ejemplos:

In [ ]:
# Ejemplo 1: Solo llamadas hechas por alguien en Maria Elena
data = Data[Data["city_a"] == 'maria elena']
data.head(15)

In [ ]:
# Ejemplo 2: Solo llamadas recibidas por alguien en Maria Elena
data = Data[Data["city_b"] == 'maria elena']
data.head(15)

In [ ]:
# Ejemplo 3: Solo llamadas de duracion igual o menor a 10 segundos
data = Data[Data["tiempo"] <= 10]
data.head(15)

In [ ]:
# Ejemplo 4: Solo llamadas hechas dentro de una ciudad
data = Data[Data["city_a"] == Data["city_b"]]
data.head(15)

In [ ]:
# Ejemplo 5: Solo llamadas hechas dentro de El Melon
data = Data[(Data["city_a"] == Data["city_b"]) & (Data["city_a"]=='el melon')]
data.head(15)

### 1. Creación de red simple
Un caso simple que podemos hacer es generar redes **no dirigidas, sin peso y sin enlaces paralelos** (es decir, un único enlace entre cada par de de nodos, independiente de la cantidad de llamadas que presenten). Este fue el tipo de red que fue utilizado para la redacción del capítulo del libro.

In [ ]:
# Creamos la red vacia
# - La opcion "directed" nos permite definir si es dirigida o no.
# - Por defecto: directed=True
net = gt.Graph(directed=False)

In [ ]:
# Creamos red con todos los datos

# Opcion "hashed=T": Indices de vertices asignados por orden de lectura
# Opcion "string_vals=T": Nombre de vertices son textos tomados desde base de datos
# (Permite asignar como nombre del vertice el ID asignado por TI+D)
edgeList = zip(Data['num_a'],Data['num_b']) # Creamos la lista de enlaces
net.add_edge_list(edgeList,hashed=True,string_vals=True)

# Ya tenemos lista nuestra red! Sin embargo, eliminaremos los enlaces redundantes (paralelos)
gt.remove_parallel_edges(net)

<div class="alert alert-block alert-info">
<b>NOTA:</b> Graph-tool permite varias maneras de agregar nodos y enlaces a una red. En el ejmplo anterior le dimos directamente una lista con todos los enlaces, usando el método <i>add_edge_list()</i>. Otras formas de crear redes son:
 <ul>
  <li>Agregar los enlaces de uno en uno, también con el método <i>add_edge_list()</i> (Ideal para pequeñas redes de prueba).</li>
    <li>Agregar un conjunto de enlaces desde un archivo <i>CSV</i>, usando el método <i>load_graph_from_csv()</i> (Esta fue la forma que usé en mi librería).</li>
    <li>Cargar una red completa desde un archivo, usando la función <i>load_graph()</i>.</li>
</ul> 
</div>

In [ ]:
# Observemos algunas caracteristicas de nuestra red:

print('========================================')
print('Numero de nodos: {:d}'.format(net.num_vertices()))
print('Numero de enlaces: {:d}'.format(net.num_edges()))

conectancia = 2*net.num_edges()/float(net.num_vertices()*(net.num_vertices()-1))
print('Conectancia: {:.3f}'.format(conectancia))

lsLocalCluster = gt.local_clustering(net)
avgClustering = gt.vertex_average(net,lsLocalCluster)[0]
print('Clustering local promedio (W&S): {:.3f}'.format(avgClustering))

print('Transitividad (Newman): {:.3f}'.format(gt.global_clustering(net)[0]))
print('Asortatividad: {:.3f}'.format(gt.assortativity(net,'out')[0]))

print('Numero de componentes conectados: {:d}'.format(len(gt.label_components(net)[1])))

# Observemos algunas caracteristicas de su componente mayor:
print('========================================')
Comp = gt.label_largest_component(net) # Componente principal
subnet = gt.GraphView(net, vfilt=Comp)
print('Nodos Componente mayor: {:d}'.format(subnet.num_vertices()))
print('Enlaces Componente mayor: {:d}'.format(subnet.num_edges()))

conectancia = 2*subnet.num_edges()/float(subnet.num_vertices()*(subnet.num_vertices()-1))
print('Conectancia Componente mayor: {:.3f}'.format(conectancia))

lsLocalCluster = gt.local_clustering(subnet)
avgClustering = gt.vertex_average(subnet,lsLocalCluster)[0]
print('Clustering local promedio (W&S): {:.3f}'.format(avgClustering))

print('Transitividad (Newman): {:.3f}'.format(gt.global_clustering(subnet)[0]))
print('Asortatividad: {:.3f}'.format(gt.assortativity(subnet,'out')[0]))

### 2. Creación de red: Enlaces con peso

Ahora repetiremos el procedimiento anterior para crear redes con peso. En realidad, _graph-tool_ no asigna un "peso" a los enlaces. Lo que hace en es asignar propiedades a los enlaces, que pueden ser numericas o no. Esto presenta tres ventajas:
1. Permite asignar como "pesos" a los enlaces distintas propiedades de la interacción.
1. Como se mencionó anteriormente, permite asignar a los enlaces propiedades no numéricas.
1. Unifica y uniforma el procedimiento de asignación de propiedades para enlaces, nodos y la red misma.

Crearemos una red donde el **peso** de cada enlace corresponde al **tiempo** de la llamada en segundos, usando el mismo procedimiento anterior. Esta vez, haremos una **red dirigida** y **sin eliminar los enlaces paralelos**.

<div class="alert alert-block alert-info">
<b>NOTA:</b> Graph-tool hace una distinción entre propiedades (en general) y propiedades internas. Las propiedades son una forma de asociar información adicional a los nodos, enlaces y al grafo mismo. Las propiedades internas son propiedades "internalizadas" dentro de una red. Esto sgnifica que pueden ser copiadas y guardadas en un archivo junto a la red. Durante el resto del minitutorial, trabajaremos con propiedades internas.
</div>

In [ ]:
# Red vacia dirigida
net = gt.Graph(directed=True)

# Creamos la propiedad que utilizaremos como tiempo:
segundos = net.new_edge_property("int")
# Ingresamos en el atributo de propiedades de enlaces de la red nuestra nueva propiedad, con la clave "Tiempo".
net.edge_properties["Tiempo"] = segundos

# Lista con las propiedades de los enlaces que vamos a agregar
lsprops = [segundos]

# Crea la red a partir de base de datos
edgeList = zip(data['num_a'],data['num_b'],data['tiempo'])
net.add_edge_list(edgeList,hashed=True,string_vals=True,eprops=lsprops)

Revisemos que hace cada línea:

`segundos = net.new_edge_property("int")`
- Creamos una propiedad de los enlaces usando el metodo _new_edge_property()_ de la red.
- Declaramos el tipo de variable que compone la propiedad como argumento del metodo.
- Dicha propiedad la guardamos en una variable llamada "_segundos_".

`net.edge_properties["Tiempo"] = tiempo`
- La red "_net_" tiene un atributo estilo diccionario para las propiedades de enlaces, nodos y de la red misma.
- Con la línea anterior, creamos una entrada de nombre "Tiempo" en el atributo de propiedades de enlaces "`edge_properties()`". Le asignamos la propiedad (vacía) almacenada en la variable "_segundos_".

`lsprops = [tiempo]`
- Es una lista con todas las propiedades que le agregaremos a los enlaces al crear la red, en el orden que vienen almacenados en la tupla de enlaces (ver más adelante). En este caso, solo creamos una propiedad, por lo que esta lista tiene un elemento.

`edgeList = zip(data['num_a'],data['num_b'],data['tiempo'])`
- Creamos una lista de enlaces como tuplas de la forma (v_in, v_out, p0, p1, ..., pn), donde pi es la i-ésima propiedad en la lista que creamos anteriormente. En este caso, nuestras tuplas tienen sólo tres elementos.

`net.add_edge_list(edgeList,hashed=True,string_vals=True,eprops=lsprops)`
- Idéntico al uso que le dimos en el ejemplo anterior, excepto porque entregamos como argumento una lista con las propiedades.
- El método `add_edge_list()` reconoce automáticamente los dos primeros elementos de cada tupla de enlaces como los vértices a enlazar, y los demás elementos como propiedades de los enlaces.

A continuación, podemos ver las distintas propiedades de nuestras red:

In [ ]:
# Lista con todas las propiedades
net.list_properties()

### 3. Creación de red: Más propiedades

Ahora repetiremos el procedimiento anterior para crear redes con nuevas propiedades. Nuevamente, haremos una **red dirigida** y **sin eliminar los enlaces paralelos**. Esta vez, nuestros enlaces serán creados con dos propiedades: **tiempo** y **dia de la llamada**. A su vez, asignaremos a los nodos los **ID** de anonimización asignados por Telefónica I+D a sus usuarios.

In [ ]:
# Red vacia dirigida
net = gt.Graph(directed=True)

# Creamos las propiedades de enlaces:
segundos = net.new_edge_property("int")
dias = net.new_edge_property("string") # OJO: Usar "str" nos lanzara un ValueError
# Ingresamos en el atributo de propiedades de enlaces de la red nuestra nueva propiedad, con la clave "Tiempo".
net.edge_properties["Tiempo"] = segundos
net.edge_properties["Fechas"] = dias

# Lista con las propiedades de los enlaces que vamos a agregar
lsprops = [segundos, dias]

# Crea la red a partir de base de datos
edgeList = zip(data['num_a'],data['num_b'],data['tiempo'],data['dia'])
# Notar que ahora guardamos en una variable el retorno de add_edge_list()
nombres = net.add_edge_list(edgeList,hashed=True,string_vals=True,eprops=lsprops)
# Esta variable contiene una propiedad de vertices, la cual almacenamos como 'ID'
net.vertex_properties['ID'] = nombres

Hagamos un repaso:
- Las propiedades de enlaces "_Tiempo_" y "_Fechas_" son añadidas de la misma forma que en el ejemplo anterior. Sólo ampliamos la lista de propiedades.
- La propiedad de nodos "*ID*" obtiene los nombres a partir de la propiedad que retorna el método `add_edge_list()`. Las cadenas de texto que usaremos como etiquetas para los nodos las obtiene con el argumento `string_vals=True`.

Revisamos las propiedades de nuestra red:

In [ ]:
# Lista con todas las propiedades
net.list_properties()

### 4. Asignación de nuevas propiedades
Al crear nuestra red, inmediatamente definimos dos propiedades para enlaces ("_Tiempo_" y "_Fechas_") y una propiedad para nodos ("*ID*"). Sin embargo, una vez la red ya ha sido creada, podemos seguir añadiendo nuevas propiedades. En este nuevo ejemplo, añadiremos una propiedad al grafo: **Ciudad**, dos propiedades nuevas a los nodos: **Tiempo_Emisor**, **Tiempo_Receptor** y **Tiempo_Total**, correspondientes al tiempo que invierte cada usuario en realizar y responder llamadas y en ambas acciones juntas; y dos nuevas propiedades a los enlaces: antenas de **Origen** y **Destino**.

Para añadir una propiedad del grafo, el procedimiento es similar a la asignación de propiedad de enlaces que vimos con anterioridad:

In [ ]:
# Creamos nueva propiedad, definiendo el tipo de variable que la compone
nombreUrbe = net.new_graph_property('string')
# Asignamos nombre a la propiedad dentro de los atributos de la red
net.graph_properties['Ciudad'] = nombreUrbe
# Asignamos valores a la propiedad
net.graph_properties['Ciudad'] = 'El Melon'

Para añadir una nueva propiedad a los nodos y enlaces, se repite el proceso, pero con el siguiente cuidado:
1. Se crea la nueva propiedad
2. Se asigna los valores de la propiedad a cada nodo/enlace uno a uno:
    - `propiedad[nodo/enlace] = valor`
3. Se guarda la nueva propiedad, con los valores ya asignados, dentro de la red

In [ ]:
# Creamos un arreglo con el tiempo total que dedica cada usuario a realizar llamadas
t_emisor=np.array([sum(data['tiempo'][data['num_a'] == net.vp.ID[v]]) for v in net.vertices()])

# Creamos nueva propiedad, definiendo el tipo de variable que la compone
tiempoLlamando = net.new_vertex_property('int')
# Asignamos valores a la propiedad
for v, t in izip(net.vertices(),t_emisor):
    tiempoLlamando[v] = t
# Asignamos la propiedad dentro de los atributos de la red
net.vertex_properties['Tiempo_Emisor'] = tiempoLlamando

In [ ]:
# Creamos nueva propiedad, definiendo el tipo de variable que la compone
id_antena_A = net.new_edge_property('string')
# Asignamos valores a la propiedad
for e, ant in izip(net.edges(),data['antena_a']):
    id_antena_A[e] = ant
# Asignamos la propiedad dentro de los atributos de la red
net.edge_properties['Origen'] = id_antena_A

Los puntos 2 y 3 se pueden invertir:

In [ ]:
# Creamos nueva propiedad, definiendo el tipo de variable que la compone
id_antena_B = net.new_edge_property('string')
# Asignamos la propiedad dentro de los atributos de la red
net.edge_properties['Destino'] = id_antena_B
# Asignamos valores a la propiedad
for e, ant in izip(net.edges(),data['antena_b']):
    net.edge_properties['Destino'][e] = ant

En el caso de propiedades con tipos escalares numéricos, se pueden realizar las asignaciones directamente desde un arreglo, usando el método `get_array()` o el atributo `a` (ambas formas parecen ser equivalentes):

In [ ]:
# Creamos un arreglo con el tiempo total que dedica cada usuario a responder llamadas
t_receptor=np.array([sum(data['tiempo'][data['num_b'] == net.vp.ID[v]]) for v in net.vertices()])
# Creamos un arreglo con el tiempo total que dedica cada usuario a llamadas
t_total = t_receptor+t_emisor

# Creamos nueva propiedad, definiendo el tipo de variable que la compone
tiempoContestando = net.new_vertex_property('int')
tiempoHablando = net.new_vertex_property('int')
# Asignamos valores a la propiedad
tiempoContestando.get_array()[:] = t_receptor
tiempoHablando.a = t_total
# Asignamos la propiedad dentro de los atributos de la red
net.vertex_properties['Tiempo_Receptor'] = tiempoContestando
net.vertex_properties['Tiempo_Total'] = tiempoHablando

Finalmente, hacemos una última revisión a nuestra lista de propiedades:

In [ ]:
net.list_properties()

### 5. Salvar y cargar la red
Podemos guardar nuestra red en archivos de distintos formatos. El recomendado es el formato `.gt`, ya que almacena las redes en un archivo binario, almacenando también las propiedades asignadas a la red, pero es incompatible con otros _softwares_. El otro formato que recomienda el creador de _graph-tool_ es `graphml`, el cual también preserva las propiedades y es compatible con otros _softwares_, pero guarda las redes en archivos de texto.

Para guardar, basta con usar el método `save()` de la red, mientras que para cargar se ocupa la función `load_graph()`.

In [ ]:
# Guardamos nuestra red
net.save('elMelonProp.gt')

# Cargamos la red con otro nombre
red = load_graph('elMelonProp.gt')

### 6. Trabajando con propiedades
<div class="alert alert-block alert-danger">
<b>EN CONSTRUCCIÓN:</b> Trabajar con la información de los pesos de los enlaces (o con cualquier propiedad en general) es algo que todavía estoy aprendiendo. Esta es una muestra limitada a mi experiencia personal.
</div>

Podemos recuperar la información de las propiedades como arreglos. Esto es válido solo si los tipos son escalares numéricos:

In [ ]:
# Guardar propiedad como arreglo (valido solo con propiedades con tipos escalares numericos)
# (Funciona como cualquier arreglo de numpy)
propArray_Tiempo_a = net.ep.Tiempo.a
propArray_Tiempo_get = net.ep.Tiempo.get_array()[:]

Podemos recuperar el valor de cualquier propiedad de un enlace *e* o nodo *v* mediante:
- `grafo.ep.["Propiedad"][e]` (para los enlaces)
- `grafo.vp.["Propiedad"][v]` (para los nodos)

El mismo resultado se puede obtener mediante:
- `grafo.ep.Propiedad[e]` (para los enlaces)
- `grafo.vp.Propiedad[v]` (para los nodos)
siempre y cuando el nombre de la propiedad no contenga espacios.

In [ ]:
# Seleccionamos tres nodos y obtenemos sus propiedades
tresNodos = [v for v in net.vertices()][:3]

for n in tresNodos:
    # Extraemos informacion
    nombre = net.vp.ID[n]
    tiempoE = net.vp.Tiempo_Emisor[n]
    tiempoR = net.vp.Tiempo_Receptor[n]
    tiempoT = net.vp.Tiempo_Total[n]
    # Imprimimos en pantalla
    print('---------------------------')
    print('El nodo llamado {}'.format(nombre))
    print('Llama {:d} segundos.'.format(tiempoE))
    print('Contesta {:d} segundos.'.format(tiempoR))
    print('Por lo tanto, habla {:d} segundos.'.format(tiempoT))


In [ ]:
# Seleccionamos tres enlaces y obtenemos sus propiedades
tresEnlaces = [e for e in net.edges()][:3]

for e in tresEnlaces:
    # Extraemos informacion
    desde = net.ep.Origen[e]
    hacia = net.ep.Destino[e]
    tiempo = net.ep.Tiempo[e]
    fecha = net.ep.Fechas[e]
    # Imprimimos en pantalla
    print('---------------------------')
    print('Una de las llamadas hecha el {}'.format(fecha))
    print('tuvo una duracion de {:d} segundos.'.format(tiempo))
    print('Se hizo desde {}'.format(desde))
    print('y fue recibida en {}.'.format(hacia))


In [ ]:
# Queremos saber cuales son los vecinos de los tres nodos escogidos
for n in tresNodos:
    # Seleccionamos los vecinos
    vecinos = n.out_neighbors()
    # Extraemos los nombres
    print('Vecinos de {}'.format(net.vp.ID[n]))
    for v in vecinos:
        print(net.vp.ID[v])

### A.0 Problemas no resueltos
<div class="alert alert-block alert-danger">
Hay algunos problemas con los que me he topado y que al momento de escribir el tutorial desconozco su solución.
</div>
- PÉRDIDA DE PROPIEDADES AL FILTRAR NODOS: Al filtrar nodos de una red usando la función `set_vertex_filter()`, la subred creada pierde las propiedades de la red original. Falta probar que pasa si se usa la función `GraphView()` (Ver referencias). Posible _bug_.
    
- OBTENCIÓN DE PROPIEDADES NO NUMÉRICAS: No sé como obtener un arreglo de propiedades si estas no son numéricas, más allá de hacer un ciclo, extrayendo las propiedades para todos los nodos/enlaces.

- COMPRESIÓN DE ENLACES MÚLTIPLES: Si hay enlaces múltiples entre dos nodos, hay que resolver como eliminar los enlaces redundantes manteniendo información de las propiedades. Posible solución manejando tabla _pandas_.


### A.1 Referencias:

#### Usadas en la creación de este tutorial:

"Official documentation: Quick start"
https://graph-tool.skewed.de/static/doc/quickstart.html

"Official documentation: Module documentation"
https://graph-tool.skewed.de/static/doc/graph_tool.html

"How to add properties to edges when using add edge list in graph tool"
https://stackoverflow.com/questions/45821741/

#### Tópicos no vistos en este tutorial

"How to access properties of a copied graph"
https://stackoverflow.com/questions/44646647/

"Setting default property value in graph tool"
https://stackoverflow.com/questions/36153290/

"Weighted degree distribution in python's graph tool"
https://stackoverflow.com/questions/27952655/

"graph tool surprisingly slow compared to networkx" (Sobre _performance_ )
https://stackoverflow.com/questions/36193773/

"graph tool access vertex properties" (Sobre _performance_ - Avanzado)
https://stackoverflow.com/questions/42787443/